# Archived code
This notebook contains code for variables that I removed from the model.

## Previous Week Averages
Note: in add_pwa first pwa columns are removed twice when calculating pwa for t and co

In [ ]:

# function for calculating the pwa of a column (excludes the day measured in the t and co columns)
def calc_pwa(data, col, pwa):
    pwa_data = pd.DataFrame()
    
    # shifting the data for the number of days and creating a new column for each day
    for i in range(1, pwa+1):
        pwa_data[col + '_' + str(i)] = data.shift(i)[col]
    
    # saving the row means as a new x_pwa column in the original dataframe
    data[col + '_pwa'] = pwa_data.aggregate(np.mean, axis=1)
    
    return data


# interface that makes calls to add_pwa depending on whether only one or all cities are used
def add_pwa(data, col, pwa):
    if use_all_cities:
        # creating a unique id for every row
        data['id'] = data.city + data.index

        data_pwa = []
        names_pwa = []
        
        # iterating over all cities
        for city in unique_cities:
            subset = data.loc[data.city == city].copy()           
            subset = calc_pwa(subset, col, pwa)
            
            # saving the pwa values and rows ids for the current city
            data_pwa.extend(subset[col+'_pwa'].iloc[pwa:])
            names_pwa.extend(subset.id[pwa:])
        
        # joining the data from each city with the original dataframe and removing the unique ids
        data_pwa = pd.Series(data_pwa, index=names_pwa, name=col+'_pwa')
        data = data.join(data_pwa, how='inner', on='id')
        data.drop(columns='id', inplace=True)
    else:
        data = calc_pwa(data, col, pwa)
        data = data[pwa:]
    
    return data

## Temperature experiments

In [ ]:
counts, values = np.histogram(df_comb.t, bins=100)
values = values[:-1] + ((values[1:] - values[:-1])/2)
values = values.astype('float64')

y = np.array(pd.Series(counts).rolling(window=7).mean())
    
maxima = find_peaks(y)[0]
minima = argrelmin(y)[0]

plt.plot(values, counts)
plt.plot(values, y, c='orange', alpha=0.75)

for maximum in maxima:
    plt.plot(np.repeat(values[maximum], 2), np.array([0, y[maximum]]))
    
for minimum in minima:
    plt.plot(np.repeat(values[minimum], 2), np.array([0, y[minimum]]))
    
t_split = values[maxima][0]
print('Splitting on:', t_split)

t_split_mask = df_comb.t > t_split

df_high_t = df.loc[t_split_mask]
df_comb_high_t = df_comb.loc[t_split_mask]
dates_high_t = dates.loc[t_split_mask]

print('Using', df_comb_high_t.shape[0], 'out of', df_comb.shape[0], 'data points', '(',
      np.round(100*df_comb_high_t.shape[0]/df_comb.shape[0], 2), '% )\n')

y_train, y_test, log_y_test, X_train, X_test, dates_test = split_data(df_high_t, df_comb_high_t, dates_high_t)